In [264]:
from sklearn.datasets import load_iris
import tensorflow as tf
data = load_iris()


In [237]:
from sklearn.preprocessing import OneHotEncoder

# label one-hot 编码
ohe = OneHotEncoder()
y_bin = []
print(np.unique(data.target))
for i in np.unique(data.target):
    y_bin.append([i])
ohe.fit(y_bin)

[0 1 2]


OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [231]:
import numpy as np

def batch_iter(data, label, batch_size, num_epoch):
    data_size = len(data)
    num_batch_per_epoch = int(data_size / batch_size) + 1
    if data_size % batch_size == 0:
        num_batch_per_epoch = int(data_size / batch_size)
    for epoch in range(num_epoch):
        # with shuffle
        shuffle_indices = np.random.permutation(np.arange(data_size))
        
        shuffle_target = label[shuffle_indices]
        shuffle_data = np.array(data[shuffle_indices])
        
        # batch generation
        for batch_num in range(num_batch_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            target = []
            for a in shuffle_target[start_index: end_index]:
                target.append([a])
            yield shuffle_data[start_index: end_index, :], ohe.transform(target).toarray(), epoch

In [350]:
class MLP(object):
    def __init__(self, feature_num, num_classes):
        self.input_x = tf.placeholder(tf.float32, [None, feature_num])
        self.input_y = tf.placeholder(tf.float32, [None, num_classes])
        
        self.W = tf.Variable(tf.zeros(shape=[feature_num, num_classes]))
        self.b = tf.Variable(tf.zeros([num_classes]))
        
        # output, linear model
        y = tf.nn.softmax(tf.nn.xw_plus_b(self.input_x, self.W, self.b))
        
        # loss definition
        with tf.name_scope("loss"):
            # softmax
            entropy = tf.nn.softmax_cross_entropy_with_logits(labels=self.input_y, logits=y)
            self.loss = tf.reduce_mean(entropy)
            correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        

In [361]:
from sklearn.cross_validation import train_test_split
import datetime
from sklearn.linear_model import LogisticRegression
train_X, test_X, train_y, test_y = train_test_split(data.data, data.target, test_size=0.2, random_state=0)

# sklean logistic regression
reg = LogisticRegression(C=999999, solver="newton-cg", multi_class="multinomial")
reg.fit(train_X, train_y)

print(reg.coef_.T)

test_target = []
for i in test_y:
    test_target.append([i])
test_target = ohe.transform(test_target).toarray()

batch_size = len(train_y)
epoch_num = 1000

# new saver
# saver = tf.train.Saver()

with tf.Session() as sess:
    mlp = MLP(feature_num=4, num_classes=3)
    train_step = tf.train.GradientDescentOptimizer(0.05).minimize(mlp.loss)
    
    sess.run(tf.global_variables_initializer())
    
    for batch_xs, batch_ys, epoch in batch_iter(train_X, train_y, batch_size, epoch_num):
        # batch data
        feed_dict = {
            mlp.input_x: batch_xs,
            mlp.input_y: batch_ys
        }
        # given batch data, get result
        _, loss, accuracy = sess.run([train_step, mlp.loss, mlp.accuracy], feed_dict)
        if epoch % 50 == 0:
            time_str = datetime.datetime.now().isoformat()
            test_accuracy = sess.run(mlp.accuracy, feed_dict={mlp.input_x:test_X, mlp.input_y:test_target})
            print("{} epoch: {:g}, loss: {:g}, accuracy: {:g}, test_accuracy: {:g}".format(time_str, epoch, loss, accuracy, test_accuracy))
    
    print(mlp.W.eval())
    
    # save model
#     save_path = saver.save(sess, "./model/model.ckpt")
#     print("model saved in file: ", save_path)

[[  4.86310369  -1.11319355  -3.74991014]
 [  6.93409701  -0.56457404  -6.36952297]
 [-12.25764049   1.57709252  10.68054797]
 [ -6.26149609  -4.96174117  11.22323726]]


# 结论
0. 使用iris数据（共150条数据），80%的数据作为训练数据（120条），20%的数据作为测试数据（30条）
1. 学习率设置为0.05，大约在650次迭代后，准确率可以达到100%
2. 权重矩阵如下：
```
[[-1.1738335   0.50324905  0.6705846 ]
 [-1.279821   -0.2514371   1.5312573 ]
 [ 2.187848   -0.0102499  -2.1775951 ]
 [ 1.7318009  -0.71315885 -1.0186411 ]]
```
2. 从weight权重矩阵看：
    * 第一类与后两个特征关系较大
    * 第二类与第一个特征关系较大
    * 第三类与前两个特征关系较大
